In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
from bs4 import BeautifulSoup
import requests
import re

In [2]:
url_1_1 = "http://www.chakoteya.net/DoctorWho/1-1.htm"
page_response_1_1 = requests.get(url_1_1, timeout = 5)
page_content_1_1 = BeautifulSoup(page_response_1_1.content, "html.parser")

In [3]:
page_content_1_1

<!DOCTYPE html>

<html lang="en-gb">
<head>
<meta charset="utf-8"/>
<meta content="Doctor Who, Episodes, Transcripts, An Unearthly Child, The Forest of Fear, The Cave of Skulls, The Firemaker, Doctor Who, Ian Chesterton, Barbara Wright, Susan Foreman, Za, Hur, Kal, Horg, Mother, William Hartnell" http-equiv="keywords"/>
<meta content="Doctor Who, Transcripts, Unearthly Child" name="Keywords"/>
<meta content="Doctor Who episode transcripts" name="Description"/>
<title>The Doctor Who Transcripts - An Unearthly Child</title>
</head>
<body bgcolor="#d5d5d5">
<font color="#2867d0" face="Arial, Helvetica, sans-serif" size="5"><b>An Unearthly Child</b></font><br/>
<p> <font face="Arial, Helvetica, sans-serif" size="2">
Original Airdate: 23 Nov, 1963<br/>
</font></p>
<div align="center">
<center>
<table border="0" width="85%">
<tbody>
<tr>
<td align="left" width="85%"> <font face="Arial, Helvetica, sans-serif" size="2"> (A
foggy night, and a policeman is patrolling his beat past I M Foreman's


In [4]:
textContent = []
paragraphs = page_content_1_1.find_all("p")
paragraphs
for i in range(len(paragraphs)):
    textContent.append(re.sub(r"[\r\n]+", r"", paragraphs[i].text))
textContent[8]

" (Ian and Barbara are parked up.) BARBARA: Over there. IAN: We're lucky there was no fog. I'd never have found this. BARBARA: Well, she doesn't seem to have arrived yet. I suppose we aredoing the right thing, aren't we? IAN: You can't justify curiosity. BARBARA: But her homework? IAN: A bit of an excuse, really, isn't it? I've seen far worse. Thetruth is, we're both curious about Susan and we won't be happy until weknow some of the answers. BARBARA: You can't just pass it off like that. If I thought I was justbeing a busybody, I'd go straight home. I thought you agreed she was abit of a mystery. IAN: Yes, but I think you'll find there's a very simple explanation toall this. BARBARA: Well, I don't know how you explain the fact that a fifteenyear old girl does not know how many shillings there are in a pound. IAN: Really? BARBARA: Really. She said she thought we were on the decimal system. IAN: Decimal system? "

In [5]:
list(set(re.findall(r"[A-Z]+:", textContent[8])))

['IAN:', 'BARBARA:']

In [6]:
def drop_colon(string):
    return string[:(len(string) - 1)]

In [7]:
elem_matches = []
for elem in textContent:
    matches = list(set(map(drop_colon, re.findall(r"[A-Z]+:", elem))))
    if len(matches) > 0:
        elem_matches.append(matches)

In [8]:
elem_matches

[['BARBARA', 'GIRL', 'BOY'],
 ['BARBARA', 'IAN'],
 ['BARBARA', 'SUSAN', 'IAN'],
 ['BARBARA', 'IAN'],
 ['BARBARA', 'SUSAN'],
 ['BARBARA', 'IAN'],
 ['SUSAN', 'IAN'],
 ['BARBARA', 'IAN'],
 ['SUSAN', 'IAN'],
 ['BARBARA', 'IAN'],
 ['BARBARA', 'IAN', 'DOCTOR'],
 ['BARBARA', 'DOCTOR', 'IAN', 'SUSAN'],
 ['HUR', 'MOTHER', 'ZA'],
 ['BARBARA', 'IAN', 'DOCTOR', 'SUSAN'],
 ['BARBARA', 'IAN', 'DOCTOR', 'SUSAN'],
 ['MOTHER', 'HUR', 'KAL', 'SUSAN', 'ALL', 'HORG', 'DOCTOR', 'ZA'],
 ['BARBARA', 'IAN', 'DOCTOR', 'SUSAN'],
 ['BARBARA', 'MOTHER', 'IAN', 'SUSAN', 'DOCTOR'],
 ['HUR', 'ZA'],
 ['DOCTOR', 'MOTHER'],
 ['HUR', 'ZA'],
 ['HUR', 'IAN', 'MOTHER', 'ZA'],
 ['BARBARA', 'HUR', 'IAN', 'SUSAN', 'DOCTOR', 'ZA'],
 ['KAL', 'MOTHER'],
 ['BARBARA', 'HUR', 'IAN', 'SUSAN', 'DOCTOR', 'ZA'],
 ['KAL', 'HORG'],
 ['KAL', 'HORG'],
 ['IAN', 'SUSAN'],
 ['IAN'],
 ['HUR', 'KAL', 'IAN', 'TRIBE', 'HORG', 'DOCTOR', 'ZA'],
 ['DOCTOR'],
 ['HUR', 'ZA'],
 ['BARBARA', 'SUSAN', 'IAN'],
 ['ZA'],
 ['BARBARA', 'IAN', 'SUSAN', 'DOCTOR'

In [65]:
class InteractionNode:
    def __init__(self, name):
        self.name = name
        self.adjacents = {}
        
    def addAdj(self, adj):
        if adj in self.adjacents.keys():
            self.adjacents[adj] += 1
        else:
            self.adjacents[adj] = 1      

class InteractionGraph:
    def __init__(self):
        self.nodes = set()
        self.node_names = {}
        
    def addNode(self, node):
        self.nodes.add(node)
        self.node_names[node.name] = node
        
    def removeNode(self, node):
        self.nodes.remove(node)
        del self.node_names[node.name]

In [66]:
# let's experiment with one episode
ep_1_1 = InteractionGraph()
for interaction in elem_matches:
    for char in interaction:
        print(char)
        if char not in ep_1_1.node_names:
            # edit the existing node
            new_node = InteractionNode(char)
            ep_1_1.addNode(new_node)
        old_node = ep_1_1.node_names[char]
        spec_interaction = interaction[:]
        spec_interaction.remove(char)
        if spec_interaction != None:
            for adj in spec_interaction:
                if adj in ep_1_1.node_names.keys():
                    old_node.addAdj(adj)
                    print("incr")
                else:
                    new_adj = InteractionNode(adj)
                    ep_1_1.addNode(new_adj)
                    old_node.addAdj(adj)
                    print("add")
        else:
            if ep_1_1.node_names[char] == None:
                # create a new node
                new_node = InteractionNode(char)
                # add the node to the interaction graph
                ep_1_1.addNode(new_node)
                print("new")
ep_1_1.node_names

BARBARA
add
add
GIRL
incr
incr
BOY
incr
incr
BARBARA
add
IAN
incr
BARBARA
add
incr
SUSAN
incr
incr
IAN
incr
incr
BARBARA
incr
IAN
incr
BARBARA
incr
SUSAN
incr
BARBARA
incr
IAN
incr
SUSAN
incr
IAN
incr
BARBARA
incr
IAN
incr
SUSAN
incr
IAN
incr
BARBARA
incr
IAN
incr
BARBARA
incr
add
IAN
incr
incr
DOCTOR
incr
incr
BARBARA
incr
incr
incr
DOCTOR
incr
incr
incr
IAN
incr
incr
incr
SUSAN
incr
incr
incr
HUR
add
add
MOTHER
incr
incr
ZA
incr
incr
BARBARA
incr
incr
incr
IAN
incr
incr
incr
DOCTOR
incr
incr
incr
SUSAN
incr
incr
incr
BARBARA
incr
incr
incr
IAN
incr
incr
incr
DOCTOR
incr
incr
incr
SUSAN
incr
incr
incr
MOTHER
incr
add
incr
add
add
incr
incr
HUR
incr
incr
incr
incr
incr
incr
incr
KAL
incr
incr
incr
incr
incr
incr
incr
SUSAN
incr
incr
incr
incr
incr
incr
incr
ALL
incr
incr
incr
incr
incr
incr
incr
HORG
incr
incr
incr
incr
incr
incr
incr
DOCTOR
incr
incr
incr
incr
incr
incr
incr
ZA
incr
incr
incr
incr
incr
incr
incr
BARBARA
incr
incr
incr
IAN
incr
incr
incr
DOCTOR
incr
incr
incr
SUSAN
inc

{'ALL': <__main__.InteractionNode at 0x1a1c9d47f0>,
 'BARBARA': <__main__.InteractionNode at 0x1a1ca24630>,
 'BOY': <__main__.InteractionNode at 0x1a1ca244e0>,
 'DOCTOR': <__main__.InteractionNode at 0x1a1c9049b0>,
 'GIRL': <__main__.InteractionNode at 0x1a1ca245c0>,
 'HORG': <__main__.InteractionNode at 0x1a1c9d4d68>,
 'HUR': <__main__.InteractionNode at 0x1a1c9d88d0>,
 'IAN': <__main__.InteractionNode at 0x1a1ca241d0>,
 'KAL': <__main__.InteractionNode at 0x1a1c929dd8>,
 'MOTHER': <__main__.InteractionNode at 0x1a1c9d8908>,
 'SUSAN': <__main__.InteractionNode at 0x1a1c8e17f0>,
 'TRIBE': <__main__.InteractionNode at 0x1a1c9da5f8>,
 'ZA': <__main__.InteractionNode at 0x1a1c9d8b00>}

In [69]:
print(list(ep_1_1.node_names['BARBARA'].adjacents))
ep_1_1.node_names['ALL'].adjacents

['GIRL', 'BOY', 'IAN', 'SUSAN', 'DOCTOR', 'MOTHER', 'HUR', 'ZA']


{'DOCTOR': 1, 'HORG': 1, 'HUR': 1, 'KAL': 1, 'MOTHER': 1, 'SUSAN': 1, 'ZA': 1}

In [ ]:
# generalize into utility functions


In [ ]:
# repeat this process for all episodes and doctors